In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession

In [5]:
sc = SparkSession.builder.appName("master").config("master","local").getOrCreate()

In [10]:
from pyspark.sql.functions import *

In [6]:
schema = ['customer' ,'start_loc' ,'end_loc']
data = [('c1', 'New York', 'Lima'),('c1', 'London', 'New York'),('c1', 'Lima', 'Sao Paulo'),('c1', 'Sao Paulo', 'New Delhi'),('c2', 'Mumbai', 'Hyderabad'),('c2', 'Surat', 'Pune'),('c2', 'Hyderabad', 'Surat'),('c3', 'Kochi', 'Kurnool'),('c3', 'Lucknow', 'Agra'),('c3', 'Agra', 'Jaipur'),('c3', 'Jaipur', 'Kochi')]


In [7]:
df = sc.createDataFrame(data=data, schema=schema)

In [8]:
df.show()

+--------+---------+---------+
|customer|start_loc|  end_loc|
+--------+---------+---------+
|      c1| New York|     Lima|
|      c1|   London| New York|
|      c1|     Lima|Sao Paulo|
|      c1|Sao Paulo|New Delhi|
|      c2|   Mumbai|Hyderabad|
|      c2|    Surat|     Pune|
|      c2|Hyderabad|    Surat|
|      c3|    Kochi|  Kurnool|
|      c3|  Lucknow|     Agra|
|      c3|     Agra|   Jaipur|
|      c3|   Jaipur|    Kochi|
+--------+---------+---------+



In [11]:
df1 = df.select(col("customer"), col("start_loc")).withColumn("location", lit("start_loc")).union(df.select(col("customer"), col("end_loc")).withColumn("location", lit("end_loc")))
df1.show()

+--------+---------+---------+
|customer|start_loc| location|
+--------+---------+---------+
|      c1| New York|start_loc|
|      c1|   London|start_loc|
|      c1|     Lima|start_loc|
|      c1|Sao Paulo|start_loc|
|      c2|   Mumbai|start_loc|
|      c2|    Surat|start_loc|
|      c2|Hyderabad|start_loc|
|      c3|    Kochi|start_loc|
|      c3|  Lucknow|start_loc|
|      c3|     Agra|start_loc|
|      c3|   Jaipur|start_loc|
|      c1|     Lima|  end_loc|
|      c1| New York|  end_loc|
|      c1|Sao Paulo|  end_loc|
|      c1|New Delhi|  end_loc|
|      c2|Hyderabad|  end_loc|
|      c2|     Pune|  end_loc|
|      c2|    Surat|  end_loc|
|      c3|  Kurnool|  end_loc|
|      c3|     Agra|  end_loc|
+--------+---------+---------+
only showing top 20 rows



In [17]:
df2 = df1.groupBy(col("customer"), col("start_loc")).agg(count("*").alias("count"), max("location").alias("location")).orderBy(col("customer"), col("start_loc"))
df2.show()

+--------+---------+-----+---------+
|customer|start_loc|count| location|
+--------+---------+-----+---------+
|      c1|     Lima|    2|start_loc|
|      c1|   London|    1|start_loc|
|      c1|New Delhi|    1|  end_loc|
|      c1| New York|    2|start_loc|
|      c1|Sao Paulo|    2|start_loc|
|      c2|Hyderabad|    2|start_loc|
|      c2|   Mumbai|    1|start_loc|
|      c2|     Pune|    1|  end_loc|
|      c2|    Surat|    2|start_loc|
|      c3|     Agra|    2|start_loc|
|      c3|   Jaipur|    2|start_loc|
|      c3|    Kochi|    2|start_loc|
|      c3|  Kurnool|    1|  end_loc|
|      c3|  Lucknow|    1|start_loc|
+--------+---------+-----+---------+



In [19]:
df2.filter(col("count")==1).groupBy(col("customer")).agg(max(when(col("location")=="start_loc", col("start_loc"))).alias("starting_location"), max(when(col("location")=="end_loc", col("start_loc"))).alias("ending_location")).show()

+--------+-----------------+---------------+
|customer|starting_location|ending_location|
+--------+-----------------+---------------+
|      c1|           London|      New Delhi|
|      c2|           Mumbai|           Pune|
|      c3|          Lucknow|        Kurnool|
+--------+-----------------+---------------+

